In [ ]:
import numpy as np
import pandas as pd

from sgp4.ext import rv2coe
from sgp4.earth_gravity import wgs72
from sgp4.propagation import sgp4 as sgprop
from sgp4.io import twoline2rv

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
with open('../data/catalog_20220706.txt') as F: lines = F.readlines()
L1 = list( filter( lambda X: X[0] == '1', lines ))
L2 = list( filter( lambda X: X[0] == '2', lines ))
catalog = pd.DataFrame( zip(L1,L2), columns=('line1','line2'))
catalog['tle_obj'] = data.apply( lambda R: twoline2rv( R['line1'],R['line2'],wgs72 ) , axis=1)
catalog['sv_epoch'] = data['tle_obj'].apply( lambda X: sgprop(X,0) ) 

In [ ]:
COEFIELDS = ['p','a','ecc','incl','omega','argp','nu','m','arglat','truelon','lonper']
def getCOE( sv ):
    coe = rv2coe( *sv , wgs72.mu )
    return dict( zip( COEFIELDS, coe ) )

catalog['coe'] = data['sv_epoch'].apply( lambda X: getCOE(X))
catalog['apogee'] = catalog.apply( lambda R:  R['coe']['a'] * (1 + R['coe']['ecc'] ) , axis=1)
catalog['perigee'] = catalog.apply( lambda R: R['coe']['a'] * (1 - R['coe']['ecc'] ) , axis=1)

In [ ]:
leo = catalog[ catalog['apogee'] < wgs72.radiusearthkm + 1500 ]
leo

In [ ]:
# from P_collision notebook

In [ ]:
leo.reset_index(inplace=True)
leo

In [ ]:
from numba import njit
@njit
def spatial_density_calc( a, incl, apogee, perigee, r, lam ):
    A = 1 / (2 * np.pi**3 * r * a * np.sqrt( np.sin( incl ) ** 2 - np.sin(lam) ** 2))
    B = np.sqrt( 1 / ((apogee -r ) * (r - perigee ))**2 )
    return A * B

# @njit
# def spatial_density_calc( orbit_row, r, lam ):
#     A = 1 / (2 * np.pi**3 * r * orbit_row['a'] * np.sqrt( np.sin( orbit_row['incl'] ) ** 2 - np.sin(lam) ** 2))
#     B = np.sqrt( 1 / ((orbit_row['apogee'] -r ) * (r - orbit_row['perigee'] ))**2 )
#     return A * B

def spatial_density( orbit_row, r, lam):
    if r < orbit_row['perigee'] : return 0
    if r > orbit_row['apogee']  : return 0
#     return spatial_density_calc( orbit_row, r, lam )
    try: return spatial_density_calc( orbit_row['a'],orbit_row['incl'],orbit_row['apogee'],orbit_row['perigee'],r,lam)
    except: return np.nan
    

In [ ]:
RADII = np.arange( wgs72.radiusearthkm, wgs72.radiusearthkm + 1500, 10 )
LATS  = np.radians( np.arange(-90,90,2) )
OUTMAT = np.zeros( (len(RADII), len(LATS) ) )
for i,RAD in enumerate(RADII):
    print(i)
    for j,LAT in enumerate(LATS):
        print(j,end='\r')
        tv = 0.
        for k,SAT in leo.iterrows():
            tv += spatial_density( SAT, RAD, LAT )
            OUTMAT[i,j] = tv

In [ ]:
import matplotlib.pyplot as plt
OUTMAT